<a href="https://colab.research.google.com/github/sumanmichael/optical-flow-tracker/blob/main/Pedestrian_Tracking_LK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
 
!cp "/gdrive/My Drive/3_N/"* "/content/"
!unzip *.zip
!rm -rf *.zip

In [ ]:
!cat gt/gt.txt | head -n 2

1,1,119.4853515625,165.3369140625,28.700000762939453,29.383333206176758,1,1,1.0
1,2,19.03515625,174.9033203125,26.649999618530273,23.91666603088379,1,1,1.0


In [ ]:
!pip install motmetrics line_profiler
%load_ext line_profiler

     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 102kB 5.4MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20069 sha256=6079e42b0a84718ec3e9a0c9c83ae567b3c31b3cad6b14f5c2b89bbcd66db2fb
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo
ERROR: pytest-benchmark 3.2.3 has requirement pytest>=3.8, but you'll have pytest 3.6.4 which is incompatible.


In [ ]:
%%time
import os
import cv2
import progressbar

sorted_list = sorted(os.listdir("img1"))
gray_images = [ cv2.cvtColor(cv2.imread("img1/"+img), cv2.COLOR_BGR2GRAY) \
               for img in progressbar.progressbar(sorted_list)]

100% (425 of 425) |######################| Elapsed Time: 0:00:02 Time:  0:00:02


CPU times: user 2.56 s, sys: 175 ms, total: 2.73 s
Wall time: 2.24 s


In [ ]:
gray_images[0].shape

(452, 636)

In [ ]:
%%file opticalflow_perPed.py
import cv2
import pandas as pd
import progressbar
import numpy as np
import sys
import csv
 
def opticalflowtracker(gray_images):
  img_size = (636,452)
  gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]
  gt = pd.read_csv("gt/gt.txt",names=gt_columns)
  gt.drop(["conf","x","y","z"],axis=1,inplace=True)
 
  getCentroid = lambda bbox: (bbox[0]+(bbox[2]//2),bbox[1]+(bbox[3]//2))
  params = dict(winSize  = (15,15),
              maxLevel = 4,
              criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
 
 
  tracker_output_np = np.array([],dtype=np.int32).reshape(-1,4)
  TRACKER_MODE = "LK"
 
  for ped_id in progressbar.progressbar(gt.id.unique()):
    gt_ped = gt[gt.id==ped_id].sort_values(by="frame")
 
    frame_number = gt_ped.frame.iloc[0]
    bbox = tuple(gt_ped[["bb_left","bb_top","bb_width","bb_height"]].iloc[0].values)
    prevPoints = np.array([getCentroid(bbox)],dtype=np.float32).reshape(-1,2)
 
    # prevImg = cv2.imread("img1/{}.jpg".format(str(frame_number).zfill(6)))
    # prevImg_gray = cv2.cvtColor(prevImg, cv2.COLOR_BGR2GRAY)
    prevImg_gray = gray_images[frame_number-1]
 
    for i,frame_number in enumerate(gt_ped.frame.iloc[1:]):
      # currImg = cv2.imread("img1/{}.jpg".format(str(frame_number).zfill(6)))
      # currImg_gray = cv2.cvtColor(currImg, cv2.COLOR_BGR2GRAY)
      currImg_gray=gray_images[frame_number-1]
 
      currPoints, st, err = cv2.calcOpticalFlowPyrLK(prevImg_gray, currImg_gray, prevPts=prevPoints, nextPts=None, **params)
      for (x,y) in currPoints:
        if(x<0 or y<0 or x>img_size[0] or y>img_size[1]):
          break
        tracker_output_np = np.concatenate((tracker_output_np,[[ int(x) for x in [frame_number,ped_id,x,y]]]),axis=0)
      if(len(prevPoints)==0):
        break
 
      prevImg_gray=currImg_gray
      prevPoints = currPoints
      
  with open(f"tracker_LK_perPed.csv", mode="w") as csvfile:
    writer = csv.writer(csvfile,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["frame","id","x","y"])
    writer.writerows(tracker_output_np)
  
# print(f"MODEL:{TRACKER_MODE}")
# %time
# opticalflowtracker(gray_images)

Writing opticalflow_perPed.py


In [ ]:
from opticalflow_perPed import opticalflowtracker
 
%lprun -T lk_time_prof_perPed  -f opticalflowtracker opticalflowtracker(gray_images)
# !cp "tracker_LK.csv" "/gdrive/My Drive/3_N/"

100% (138 of 138) |######################| Elapsed Time: 0:00:27 Time:  0:00:27



*** Profile printout saved to text file 'lk_time_prof_perPed'. 


In [ ]:
# import pandas as pd

# gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]

# tracker_csv_df = pd.read_csv("tracker_LK.csv")
# gt_csv_df = pd.read_csv("gt/gt.txt",names=gt_columns)

In [ ]:
# gt_csv_df["h"] = gt_csv_df["bb_left"]+(gt_csv_df["bb_width"]//2)
# gt_csv_df["k"] = gt_csv_df["bb_top"]+(gt_csv_df["bb_height"]//2)
# gt_csv_df.drop(["bb_left", "bb_top", "bb_width", "bb_height","conf","x","y","z"],axis=1,inplace=True)

In [ ]:
%%file opticalflow_perFrame.py
import pandas as pd
import numpy as np
import progressbar
import cv2
import csv
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,15)

img_size = (636,452)
def opticalflowtracker(gray_images):
  gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]
  gt_csv_df = pd.read_csv("gt/gt.txt",names=gt_columns)
  gt_csv_df["h"] = gt_csv_df["bb_left"]+(gt_csv_df["bb_width"]/2)
  gt_csv_df["k"] = gt_csv_df["bb_top"]+(gt_csv_df["bb_height"]/2)
  gt_csv_df.drop(["bb_left", "bb_top", "bb_width", "bb_height","conf","x","y","z"],axis=1,inplace=True)

  params = dict(winSize  = (15,15),
              maxLevel = 2,
              criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
  ppp = dict((int(arr[1]),(arr[2],arr[3])) for arr in gt_csv_df[gt_csv_df.frame==1].values)
  prevImg_gray = gray_images[0]

  tracker_output_np = np.array([],dtype=np.int32).reshape(-1,4)

  for frame in progressbar.progressbar(range(2,max(gt_csv_df.frame.to_numpy())+1)):
    currImg_gray = gray_images[frame-1]
    prev_points = np.array(list(ppp.values()),dtype=np.float32)
    curr_points,_,_ = cv2.calcOpticalFlowPyrLK(prevImg_gray, currImg_gray, prevPts=prev_points, nextPts=None, **params)
    ppp = dict(zip(ppp.keys(),curr_points))
    rem_ppp_keys = []
    for k,v in ppp.items():
      if(v[0]<0 or v[1]<0 or v[0]>img_size[0] or v[1]>img_size[1]):
        rem_ppp_keys.append(k)
    for k in rem_ppp_keys:
      del ppp[k]
    curr_ped_ids_gt = set(gt_csv_df[gt_csv_df.frame==frame]["id"])
    curr_ped_ids_tk = set(ppp.keys())
    for id in curr_ped_ids_gt.difference(curr_ped_ids_tk):
      _,_,x,y = gt_csv_df[(gt_csv_df.id==id) & (gt_csv_df.frame==frame)].iloc[0]
      ppp[id] = np.array([x,y],dtype=np.float32)
    for k,v in ppp.items():
      tracker_output_np = np.concatenate((tracker_output_np,[[ int(x) for x in [frame,k,*v]]]),axis=0)
    
    prevImg_gray=currImg_gray
    prevPoints = curr_points

  with open(f"tracker_LK_perFrame.csv", mode="w") as csvfile:
      writer = csv.writer(csvfile,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      writer.writerow(["frame","id","x","y"])
      writer.writerows(tracker_output_np)
# opticalflowtracker(gray_images)

Writing opticalflow_perFrame3.py


In [ ]:
from opticalflow_perFrame import opticalflowtracker
 
%lprun -T lk_time_prof_perFrame  -f opticalflowtracker opticalflowtracker(gray_images)
# !cp lkprofm2 "/gdrive/My Drive/outputmot"
# !cp "tracker_LK_2.csv" "/gdrive/My Drive/outputmot"

100% (424 of 424) |######################| Elapsed Time: 0:00:04 Time:  0:00:04



*** Profile printout saved to text file 'lk_time_prof_perFrame'. 


In [ ]:
import motmetrics as mm
import progressbar
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 900

def get_datasets(tk_path,gt_path="gt/gt.txt"):
  gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]
  gt_csv_df = pd.read_csv(gt_path,names=gt_columns) 
  gt_csv_df.drop(["conf","x","y","z"],axis=1,inplace=True)

  tracker_csv_df = pd.read_csv(tk_path)
  return gt_csv_df,tracker_csv_df

def get_mm(gt_csv_df,tracker_csv_df,type,THRESH,NAME):

  METRIC_COLUMNS = ['mota', 'motp', 'num_misses','num_switches', \
                    'num_false_positives', 'num_unique_objects', 'mostly_tracked',\
                    'partially_tracked','mostly_lost']

  traj_gt = gt_csv_df.values
  traj_model = tracker_csv_df.values
  frames = np.unique(traj_gt[:,0])

  # Create an accumulator that will be updated during each frame
  acc = mm.MOTAccumulator(auto_id=True)

  # for frame in progressbar.progressbar(frames):
  for frame in frames:
      gt = traj_gt[traj_gt[:,0]==frame]
      gt_id = gt[:, 1]
      gt_coords = gt[:, 2:]

      model = traj_model[traj_model[:,0]==frame]
      model_id = model[:, 1]
      model_coords = model[:, 2:]

      if(type=="dot"):
        gt_coords[:,0] += (gt_coords[:,2]/2)
        gt_coords[:,1] += (gt_coords[:,3]/2)
        gt_coords = gt_coords[:,:2]
        C = mm.distances.norm2squared_matrix(gt_coords, model_coords, max_d2=THRESH)
      else:
        C = mm.distances.iou_matrix(gt_coords, model_coords, max_iou=THRESH)

      f_id = acc.update(
          gt_id,      # Ground truth objects in this frame
          model_id,   # Detector hypotheses in this frame
          C
      )

  mh = mm.metrics.create()
  summary = mh.compute(acc, metrics=METRIC_COLUMNS, name=NAME)
  strsummary = mm.io.render_summary(
    summary,
    # formatters=mh.formatters,
    namemap=mm.io.motchallenge_metric_names
  )
  print(strsummary)

# for tresh in range(100,10000,100):
#   print(tresh)
  # get_mm(*get_datasets(tk_path="tracker_LK_perPed.csv"),type="dot",THRESH=tresh)
get_mm(*get_datasets(tk_path="tracker_LK_2.csv"),type="dot",NAME="LK_MOT20",THRESH=1000)

              MOTA        MOTP     FN   IDs     FP   GT  MT   PT  ML
LK_MOT20 -3.760852  457.387494  17759  3864  92766  138   1  105  32


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

def get_datasets(tk_path,gt_path="gt/gt.txt"):
  gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]
  gt_csv_df = pd.read_csv(gt_path,names=gt_columns) 
  gt_csv_df.drop(["conf","x","y","z"],axis=1,inplace=True)

  tracker_csv_df = pd.read_csv(tk_path)
  return gt_csv_df,tracker_csv_df

def plot_ped_track(gt_csv_df,tracker_csv_df,PED_IDs,type):
  
  fig,ax = plt.subplots(1,figsize=(15,15))
  ax.imshow(cv2.imread("img1/{}.jpg".format(str(1).zfill(6))),cmap="gray")
  
  for PED_ID in PED_IDs:
    data_gt = gt_csv_df[gt_csv_df.id==PED_ID].values[:,2:]
    data_tk = tracker_csv_df[tracker_csv_df.id==PED_ID].values[:,2:]

    if type=="dot":
      data_gt[:,0] += (data_gt[:,2]/2)
      data_gt[:,1] += (data_gt[:,3]/2)
      data_gt = data_gt[:,:2]
      ax.scatter(data_gt[:,0],data_gt[:,1],color="red")
      ax.scatter(data_tk[:,0],data_tk[:,1],color="blue")
    else:
      for x,y,w,h in data_gt:
        ax.add_patch(
            patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
        )

      for x,y,w,h in data_tk:
        ax.add_patch(         
            patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='b',facecolor='none')
        )

def plot_frame(gt_csv_df,tracker_csv_df,FRAME_NUM,type):
  fig,ax = plt.subplots(1,figsize=(15,15))
  ax.imshow(cv2.imread("img1/{}.jpg".format(str(FRAME_NUM).zfill(6))))

  data_gt = gt_csv_df[gt_csv_df.frame==FRAME_NUM].values[:,2:]
  data_tk = tracker_csv_df[tracker_csv_df.frame==FRAME_NUM].values[:,2:]
  if type=="dot":
    data_gt[:,0] += (data_gt[:,2]/2)
    data_gt[:,1] += (data_gt[:,3]/2)
    data_gt = data_gt[:,:2]
    ax.scatter(data_gt[:,0],data_gt[:,1],color="red")
    ax.scatter(data_tk[:,0],data_tk[:,1],color="blue")
  else:
    for x,y,w,h in data_gt:
      ax.add_patch(
          patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
      )

    for x,y,w,h in data_tk:
      ax.add_patch(
          patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='b',facecolor='none')
      )

# plot_frame(*get_datasets(),201,type="box")
x,y = get_datasets(tk_path="tracker_LK_perFrame.csv")
plot_ped_track(x,y,[40],"dot")


## Time for pre-fetching and gray-scale conversion to avoid IO bottleneck (Not advisible for large dataset)

**425 Frames**
CPU times: user 2.51 s, sys: 178 ms, **total: 2.68 s**
Wall time: 2.66 s

## tracker_LK_perPed.csv

    TRESH      : 500
    num_frames : 425
    mota       : 0.45016
    motp       : 76.164364

## tracker_LK_perFrame.csv

    TRESH      : 500
    num_frames : 425
    mota       : 0.381904
    motp       : 79.370357

In [ ]:
gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]
gt_csv_df = pd.read_csv("gt/gt.txt",names=gt_columns)
gt_csv_df[["bb_width","bb_height"]].describe()

,bb_width,bb_height
count,24027.000000,24027.000000
mean,27.733652,28.963877
std,6.967582,5.902924
min,4.410156,9.800781
25%,22.904492,24.169618
50%,26.730273,28.465820
75%,32.391797,32.953960
max,49.017914,45.555557


In [ ]:
gt_columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"]
gt_csv_df = pd.read_csv("gt/gt.txt",names=gt_columns)
gt_csv_df["h"] = gt_csv_df["bb_left"]+(gt_csv_df["bb_width"]/2)
gt_csv_df["k"] = gt_csv_df["bb_top"]+(gt_csv_df["bb_height"]/2)
gt_csv_df.drop(["bb_left", "bb_top", "bb_width", "bb_height","conf","x","y","z"],axis=1,inplace=True)
gt_csv_df = gt_csv_df[gt_csv_df.id==2]
gt_csv_df

,frame,id,h,k
1,1,2,32.360156,186.861653
68,2,2,34.282031,187.117879
135,3,2,36.203906,187.374104
202,4,2,38.125781,187.630330
269,5,2,40.047656,187.886556
...,...,...,...,...
18557,295,2,618.659961,59.761067
18611,296,2,618.659961,59.761067
18665,297,2,618.659961,59.761067
18718,298,2,618.659961,59.761067
